In [1]:
import pandas as pd
import numpy as np

In [ ]:
# from geoband.API import *
# GetCompasData('SBJ_2208_001', '6', '6_연수구_세대원수별_세대수.xlsx')

[2022-09-22 14:47:25] 성공 - 6_연수구_세대원수별_세대수.xlsx


'6_연수구_세대원수별_세대수.xlsx'

In [2]:
# 세대원수 : housemate
# 2016 ~ 2022 월별 데이터

# housemate = pd.read_excel('6_연수구_세대원수별_세대수.xlsx')
housemate = pd.read_excel('/content/6_연수구_세대원수별_세대수.xlsx')
housemate.head()

,emd_nm,base_date,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,인천광역시 연수구,201607,118738,31614,21111,25647,31086,7391,1440,315,84,28,22
1,인천광역시 연수구 옥련1동,201607,8363,2265,1694,1870,1954,469,83,26,1,1,0
2,인천광역시 연수구 옥련2동,201607,8177,1813,1514,1938,2245,528,112,23,1,3,0
3,인천광역시 연수구 선학동,201607,8241,2988,1772,1524,1456,380,92,19,6,1,3
4,인천광역시 연수구 연수1동,201607,11615,5430,1998,1739,1784,531,98,26,5,3,1


# 전처리

### ⭐22.09.26 수정 !
- 해당 연도의 값은 매년 말월(12월)을 기준으로 함.

In [3]:
housemate2 = housemate.copy()

In [4]:
# 날짜값을 문자열로 변경
housemate2['base_date'] = housemate2['base_date'].astype(str)

In [5]:
# 12월만 추출하여 데이터프레임 형성

date_list = list(housemate2['base_date'])
x_december = []

for i in range(len(date_list)):
    if housemate2['base_date'][i][-2:] == '12':
        # i 는 12월 값의 인덱스 번호
        x_december.append(i)

# x_december

temp = {}
for i in x_december:
    temp[i] = housemate2.loc[i]

housemate_12 = pd.DataFrame(temp).T

In [6]:
### 12월 값을 기준으로 세대수 분석

housemate_12.reset_index(inplace = True)
housemate_12 = housemate_12.drop(columns = 'index')
housemate_12.head()

,emd_nm,base_date,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,인천광역시 연수구,201612,120580,32121,21708,26208,31083,7479,1501,335,94,32,19
1,인천광역시 연수구 옥련1동,201612,8396,2341,1718,1870,1887,464,89,23,4,0,0
2,인천광역시 연수구 옥련2동,201612,8167,1795,1527,1970,2207,531,109,22,3,3,0
3,인천광역시 연수구 선학동,201612,8254,2997,1786,1531,1434,369,107,17,9,1,3
4,인천광역시 연수구 연수1동,201612,11407,5325,2005,1719,1701,536,89,23,6,2,1


In [7]:
# 연도만 추출
date_list = list(housemate_12['base_date'])
year = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']

for i in range(len(date_list)):
    for j in range(len(year)):
        if date_list[i][:-2] == year[j]:
            date_list[i] = year[j]
            
housemate_12['base_date'] = date_list

In [8]:
housemate_12['base_date'].unique()

array(['2016', '2017', '2018', '2019', '2020', '2021'], dtype=object)

In [9]:
# '인천광역시 연수구 ' 라는 값 존재
# 1동,2동 등으로 구분
housemate_12['emd_nm'].unique()

array(['인천광역시 연수구 ', '인천광역시 연수구 옥련1동', '인천광역시 연수구 옥련2동', '인천광역시 연수구 선학동',
       '인천광역시 연수구 연수1동', '인천광역시 연수구 연수2동', '인천광역시 연수구 연수3동',
       '인천광역시 연수구 청학동', '인천광역시 연수구 동춘1동', '인천광역시 연수구 동춘2동',
       '인천광역시 연수구 동춘3동', '인천광역시 연수구 송도1동', '인천광역시 연수구 송도2동',
       '인천광역시 연수구 송도3동', '인천광역시 연수구 송도4동', '인천광역시 연수구 송도5동'], dtype=object)

In [10]:
# 읍면동 명칭 수정
housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'옥련동' if x in ['인천광역시 연수구 옥련1동', 
                                                                           '인천광역시 연수구 옥련2동'] else x )

housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'연수동' if x in ['인천광역시 연수구 연수1동', '인천광역시 연수구 연수2동', 
                                                                           '인천광역시 연수구 연수3동'] else x )

housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'동춘동' if x in ['인천광역시 연수구 동춘1동', '인천광역시 연수구 동춘2동', 
                                                                           '인천광역시 연수구 동춘3동'] else x )

housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'송도동' if x in ['인천광역시 연수구 송도1동', '인천광역시 연수구 송도2동', 
                                                                           '인천광역시 연수구 송도3동', '인천광역시 연수구 송도4동', 
                                                                           '인천광역시 연수구 송도5동'] else x )

housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'선학동' if x in ['인천광역시 연수구 선학동'] else x )

housemate_12['emd_nm'] = housemate_12['emd_nm'].apply(lambda x :'청학동' if x in ['인천광역시 연수구 청학동'] else x )


In [11]:
housemate_12['emd_nm'].unique()

array(['인천광역시 연수구 ', '옥련동', '선학동', '연수동', '청학동', '동춘동', '송도동'],
      dtype=object)

In [12]:
housemate_12.head()

,emd_nm,base_date,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,인천광역시 연수구,2016,120580,32121,21708,26208,31083,7479,1501,335,94,32,19
1,옥련동,2016,8396,2341,1718,1870,1887,464,89,23,4,0,0
2,옥련동,2016,8167,1795,1527,1970,2207,531,109,22,3,3,0
3,선학동,2016,8254,2997,1786,1531,1434,369,107,17,9,1,3
4,연수동,2016,11407,5325,2005,1719,1701,536,89,23,6,2,1


In [13]:
# 값 확인
housemate_12[housemate_12['base_date'] == '2016'].groupby(['base_date','emd_nm']).sum().reset_index()
housemate_12[housemate_12['base_date'] == '2020'].groupby(['base_date','emd_nm']).sum().reset_index()

,base_date,emd_nm,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,2020,동춘동,22037,3849,4820,5707,5998,1358,237,43,15,3,7
1,2020,선학동,8037,3364,1907,1376,1048,252,70,15,4,0,1
2,2020,송도동,66092,15923,11810,16214,17632,3690,608,156,41,10,8
3,2020,연수동,27335,11911,6037,4542,3785,851,161,31,10,4,3
4,2020,옥련동,16229,4723,3835,3510,3252,755,117,29,3,4,1
5,2020,인천광역시 연수구,152266,45698,31263,33312,33055,7246,1280,293,75,24,20
6,2020,청학동,12536,5928,2854,1963,1340,340,87,19,2,3,0


# 최종 : 세대원수_전체_1821

In [14]:
housemate_1822 = housemate_12[housemate_12['base_date']>='2018']
housemate_1821 = housemate_1822[housemate_1822['base_date']<'2022']
housemate_1821.reset_index(inplace = True)
housemate_1821 = housemate_1821.drop(columns = 'index')
housemate_1821

,emd_nm,base_date,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,인천광역시 연수구,2018,129806,35700,24756,28498,31421,7430,1510,358,88,23,22
1,옥련동,2018,8356,2460,1847,1780,1731,429,84,20,1,2,2
2,옥련동,2018,7995,1853,1567,1957,1978,492,117,24,6,0,1
3,선학동,2018,8140,3144,1812,1497,1248,322,91,17,7,0,2
4,연수동,2018,10509,5025,1934,1550,1451,437,90,17,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,송도동,2021,14547,4777,2350,2911,3441,814,188,46,17,2,1
57,송도동,2021,11831,1682,1862,3174,4018,908,137,37,10,2,1
58,송도동,2021,18832,5458,3329,4487,4559,836,114,30,10,5,4
59,송도동,2021,9380,2311,1664,2405,2502,422,53,13,5,3,2


In [15]:
# 저장
housemate_1821.to_csv('최종_세대원수_전체_수정.csv', encoding = 'utf-8', index = False)

In [16]:
# 불러오기
housemate_1821 = pd.read_csv('최종_세대원수_전체_수정.csv', encoding = 'utf-8')
housemate_1821.head()

,emd_nm,base_date,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,인천광역시 연수구,2018,129806,35700,24756,28498,31421,7430,1510,358,88,23,22
1,옥련동,2018,8356,2460,1847,1780,1731,429,84,20,1,2,2
2,옥련동,2018,7995,1853,1567,1957,1978,492,117,24,6,0,1
3,선학동,2018,8140,3144,1812,1497,1248,322,91,17,7,0,2
4,연수동,2018,10509,5025,1934,1550,1451,437,90,17,3,1,1


# 최종 : 세대원수_동별_total_1인

In [17]:
dong_housemate = housemate_1821[housemate_1821['emd_nm'] != '인천광역시 연수구 ']
dong_housemate = dong_housemate.groupby(['base_date','emd_nm']).sum()
dong_housemate.reset_index(inplace = True)
dong_housemate.head()

,base_date,emd_nm,total_hshld,1p_hshld,2p_hshld,3p_hshld,4p_hshld,5p_hshld,6p_hshld,7p_hshld,8p_hshld,9p_hshld,10p_hshld
0,2018,동춘동,18764,2746,3520,4657,6065,1421,260,59,22,6,8
1,2018,선학동,8140,3144,1812,1497,1248,322,91,17,7,0,2
2,2018,송도동,45995,8838,7359,11550,14252,3179,607,153,40,10,7
3,2018,연수동,27971,11326,5767,4943,4514,1117,232,58,9,3,2
4,2018,옥련동,16351,4313,3414,3737,3709,921,201,44,7,2,3


In [18]:
dong_housemate = dong_housemate[['base_date', 'emd_nm', 'total_hshld', '1p_hshld']]
dong_housemate.head()

,base_date,emd_nm,total_hshld,1p_hshld
0,2018,동춘동,18764,2746
1,2018,선학동,8140,3144
2,2018,송도동,45995,8838
3,2018,연수동,27971,11326
4,2018,옥련동,16351,4313


In [19]:
# 1인세대비율
dong_housemate['1p_rate'] = (dong_housemate['1p_hshld'] / dong_housemate['total_hshld']) * 100

In [20]:
dong_housemate.head()

,base_date,emd_nm,total_hshld,1p_hshld,1p_rate
0,2018,동춘동,18764,2746,14.634406
1,2018,선학동,8140,3144,38.624079
2,2018,송도동,45995,8838,19.215132
3,2018,연수동,27971,11326,40.491938
4,2018,옥련동,16351,4313,26.377592


In [21]:
# 저장
dong_housemate.to_csv('최종_세대원수_동별_1821_수정.csv', encoding = 'utf-8', index = False)

In [22]:
# 불러오기
dong_housemate = pd.read_csv('최종_세대원수_동별_1821_수정.csv', encoding = 'utf-8')
dong_housemate.head()

,base_date,emd_nm,total_hshld,1p_hshld,1p_rate
0,2018,동춘동,18764,2746,14.634406
1,2018,선학동,8140,3144,38.624079
2,2018,송도동,45995,8838,19.215132
3,2018,연수동,27971,11326,40.491938
4,2018,옥련동,16351,4313,26.377592


# 최종 : 세대원수_신도심_원도심

In [ ]:
# 원도심
housemate_old = dong_housemate[dong_housemate['emd_nm'] != '송도동']
housemate_old = housemate_old.groupby('base_date').sum()
housemate_old.reset_index(inplace = True)
housemate_old.insert(1, 'center', '원도심')
housemate_old

,base_date,center,total_hshld,1p_hshld
0,2018,원도심,83811,26862
1,2019,원도심,85871,28173
2,2020,원도심,86174,29775
3,2021,원도심,86444,30918


In [ ]:
# 원도심
housemate_new = dong_housemate[dong_housemate['emd_nm'] == '송도동']
housemate_new = housemate_new.groupby('base_date').sum()
housemate_new.reset_index(inplace = True)
housemate_new.insert(1, 'center', '신도심')
housemate_new

,base_date,center,total_hshld,1p_hshld
0,2018,신도심,45995,8838
1,2019,신도심,54829,12385
2,2020,신도심,66092,15923
3,2021,신도심,69540,17496


In [ ]:
# 합치기
housemate_old_new = pd.concat([housemate_new, housemate_old])
housemate_old_new = housemate_old_new.sort_values(by = housemate_old_new.columns[0])
housemate_old_new.reset_index(inplace = True)
housemate_old_new = housemate_old_new.drop(columns = 'index')
housemate_old_new

,base_date,center,total_hshld,1p_hshld
0,2018,신도심,45995,8838
1,2018,원도심,83811,26862
2,2019,신도심,54829,12385
3,2019,원도심,85871,28173
4,2020,신도심,66092,15923
5,2020,원도심,86174,29775
6,2021,신도심,69540,17496
7,2021,원도심,86444,30918


In [ ]:
# 저장
housemate_old_new.to_csv('최종_세대원수_신도심_원도심_수정.csv', encoding = 'utf-8', index = False)

In [ ]:
# 불러오기
housemate_old_new = pd.read_csv('최종_세대원수_신도심_원도심_수정.csv', encoding = 'utf-8')
housemate_old_new

,base_date,center,total_hshld,1p_hshld
0,2018,신도심,45995,8838
1,2018,원도심,83811,26862
2,2019,신도심,54829,12385
3,2019,원도심,85871,28173
4,2020,신도심,66092,15923
5,2020,원도심,86174,29775
6,2021,신도심,69540,17496
7,2021,원도심,86444,30918


# 데이터 요청

In [ ]:
# from geoband.API import *
# RequestCompasData('최종_세대원수_신도심_원도심.csv')
# RequestCompasData('최종_세대원수_동별_1821.csv')
# RequestCompasData('최종_세대원수_전체.csv')